In [12]:
pip install transformers torch 

     ---------------------------------------- 0.0/43.7 kB ? eta -:--:--
     ----------------------------------- -- 41.0/43.7 kB 991.0 kB/s eta 0:00:01
     -------------------------------------- 43.7/43.7 kB 711.0 kB/s eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
    --------------------------------------- 0.2/9.4 MB 2.4 MB/s eta 0:00:04
   - -------------------------------------- 0.3/9.4 MB 2.7 MB/s eta 0:00:04
   -- ------------------------------------- 0.6/9.4 MB 3.2 MB/s eta 0:00:03
   --- ------------------------------------ 0.8/9.4 MB 3.7 MB/s eta 0:00:03
   ---- ----------------------------------- 1.1/9.4 MB 4.1 MB/s eta 0:00:03
   ----- ---------------------------------- 1.4/9.4 MB 4.4 MB/s eta 0:00:02
   ------ ------------


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\tranl\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


# Load data

In [28]:
import pandas as pd

df = pd.read_csv('../csv/processed_medical.csv')
df


,Title,Detailed Content,Reference Link
0,rào_cản tự_nhiên chống lại nhiễm_trùng da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,https://www.msdmanuals.com/vi-vn/professional/...
1,rào_cản tự_nhiên chống lại nhiễm_trùng niêm_mạc,nhiều màng_nhầy ngập trong chất tiết có đặc_tí...,https://www.msdmanuals.com/vi-vn/professional/...
2,rào_cản tự_nhiên chống lại nhiễm_trùng đường h...,đường hô_hấp có các hệ_thống lọc đường thở trê...,https://www.msdmanuals.com/vi-vn/professional/...
3,rào_cản tự_nhiên chống lại nhiễm_trùng đường t...,các rào_cản trong đường tiêu_hóa bao_gồm ph ac...,https://www.msdmanuals.com/vi-vn/professional/...
4,rào_cản tự_nhiên chống lại nhiễm_trùng đường s...,các rào_cản về đường sinh_dục tiết_niệu bao_gồ...,https://www.msdmanuals.com/vi-vn/professional/...
...,...,...,...
22206,tiêu_chí đánh_giá và theo_dõi hồi_sức bằng dịc...,vì lượng nước_tiểu không cung_cấp chỉ từng phú...,https://www.msdmanuals.com/vi-vn/professional/...
22207,tiêu_chí đánh_giá và theo_dõi hồi_sức bằng dịc...,bệnh_nhân bị sốc mất máu do chấn_thương có_thể...,https://www.msdmanuals.com/vi-vn/professional/...
22208,áp_lực tĩnh_mạch trung_tâm,vì lượng nước_tiểu không cung_cấp chỉ từng phú...,https://www.msdmanuals.com/vi-vn/professional/...
22209,sốc mất máu do chấn_thương,bệnh_nhân bị sốc mất máu do chấn_thương có_thể...,https://www.msdmanuals.com/vi-vn/professional/...


# Xử lý các giá trị null và duplicate

In [29]:
df.isnull().sum()

Title               0
Detailed Content    0
Reference Link      0
dtype: int64

In [30]:
# Kiểm tra số lượng giá trị trùng lặp trong cột 'content'
duplicates_count = df['Detailed Content'].duplicated().sum()
print(f"Số lượng giá trị trùng lặp trong cột 'content': {duplicates_count}")

Số lượng giá trị trùng lặp trong cột 'content': 7196


In [31]:
# Xóa các giá trị trùng lặp trong cột 'content'
df = df.drop_duplicates(subset='Title', keep='first')
df = df.drop_duplicates(subset='Detailed Content', keep='first')

In [32]:
df = df.drop_duplicates()

In [33]:
duplicates_count = df.duplicated().sum()
print(f"Số lượng giá trị trùng lặp trong cột 'content': {duplicates_count}")

Số lượng giá trị trùng lặp trong cột 'content': 0


# Tìm các giá trị trùng lặp

In [17]:
# Tìm các hàng trùng lặp
duplicates = df[df.duplicated(subset='Detailed Content', keep='first')]

# Hiển thị các hàng trùng lặp
print("Các hàng trùng lặp:")
duplicates

Các hàng trùng lặp:


,Title,Detailed Content,Reference Link


In [9]:
# # Tìm các hàng trùng lặp với hàng số 5 (index = 4)
# duplicate_rows = df[df['Detailed Content'] == df.loc[5, 'Detailed Content']]

# # Hiển thị kết quả
# duplicate_rows

# Lưu data

In [34]:
# Lưu DataFrame vào file CSV
df.to_csv('../csv/processed_medical.csv', index=False, encoding='utf-8')

# Show detail dataset

In [20]:
# # Lấy nội dung của cột "Content" ở dòng đầu tiên
# first_row_content = df.loc[5, 'Detailed Content']

# # In nội dung của cột "Content" ở dòng đầu tiên
# print("Content of the first row:")
# print(first_row_content)

In [35]:
df = pd.read_csv('../csv/processed_medical.csv')
df

,Title,Detailed Content,Reference Link
0,rào_cản tự_nhiên chống lại nhiễm_trùng da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,https://www.msdmanuals.com/vi-vn/professional/...
1,rào_cản tự_nhiên chống lại nhiễm_trùng niêm_mạc,nhiều màng_nhầy ngập trong chất tiết có đặc_tí...,https://www.msdmanuals.com/vi-vn/professional/...
2,rào_cản tự_nhiên chống lại nhiễm_trùng đường h...,đường hô_hấp có các hệ_thống lọc đường thở trê...,https://www.msdmanuals.com/vi-vn/professional/...
3,rào_cản tự_nhiên chống lại nhiễm_trùng đường t...,các rào_cản trong đường tiêu_hóa bao_gồm ph ac...,https://www.msdmanuals.com/vi-vn/professional/...
4,rào_cản tự_nhiên chống lại nhiễm_trùng đường s...,các rào_cản về đường sinh_dục tiết_niệu bao_gồ...,https://www.msdmanuals.com/vi-vn/professional/...
...,...,...,...
11989,dịch để hồi_sức bằng dịch qua đường tĩnh_mạch ...,1 transfusion of plasma platelets and red bloo...,https://www.msdmanuals.com/vi-vn/professional/...
11990,đường và tỷ_lệ dịch,đường và tỷ_lệ dịch các ống thông iv ngoại_vi ...,https://www.msdmanuals.com/vi-vn/professional/...
11991,tiêu_chí đánh_giá và theo_dõi hồi_sức bằng dịc...,vì lượng nước_tiểu không cung_cấp chỉ từng phú...,https://www.msdmanuals.com/vi-vn/professional/...
11992,tiêu_chí đánh_giá và theo_dõi hồi_sức bằng dịc...,bệnh_nhân bị sốc mất máu do chấn_thương có_thể...,https://www.msdmanuals.com/vi-vn/professional/...


In [36]:
# Lấy nội dung của cột "Content" ở dòng đầu tiên
first_row_content = df.loc[1557, 'Detailed Content']

# In nội dung của cột "Content" ở dòng đầu tiên
print("Content of the first row:")
print(first_row_content)

Content of the first row:
trong một_số trường_hợp các thuốc chống ngưng tập tiểu_cầu ví_dụ aspirin bị dừng lại từ 5 đến 7 ngày_trước khi phẫu_thuật tuy_nhiên ở những bệnh_nhân đặt stent động_mạch_vành việc ngừng điều_trị chống kết tập tiểu_cầu dù là aspirin hay liệu_pháp kháng tiểu_cầu kép đều làm tăng nguy_cơ bị huyết khối lều vành có_thể gây tử_vong nguy_cơ này có_thể lớn hơn nguy_cơ chảy_máu phẫu_thuật và các quyết_định liên_quan đến việc quản_lý các phác_đồ một thuốc hoặc hai thuốc nên được cá_nhân_hóa với ý_kiến đóng_góp của một nhóm đa ngành các yếu_tố cần xem_xét bao_gồm loại stent trần hoặc rửa giải bằng thuốc thời_gian kể từ khi đặt loại phẫu_thuật và liệu một thủ_thuật tự chọn có_thể được hoãn lại cho đến khi các giai_đoạn tăng nguy_cơ trôi qua hay không 1 ngoại_trừ một_số tiểu thủ_thuật warfarin được ngừng 5 ngày_trước khi phẫu_thuật inr tỷ_lệ chuẩn_hóa quốc_tế tại thời_điểm phẫu_thuật phải ≤ 15 các bệnh_nhân có nguy_cơ cao xơ_vữa động_mạch ví_dụ bệnh_nhân có tiền_sử tắc_ngh